This notebook shows how to use [dvc](https://dvc.org/) [experiments](https://github.com/iterative/dvc/wiki/Experiments) in model development. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image. The model is built in [pytorch](https://pytorch.org/) as convolutional neural network with a simplified architecture, which should be able to quickly run on most computers.

### Getting started

To get started, clone this repository and navigate to it.

The only other prerequisite is [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/). Once conda is installed, create a virtual environment from the existing `environment.yaml` file and activate it:

```bash
conda env create -f environment.yml
conda activate dvc
```

If you want to run this notebook directly, do so after activating the conda environment.

### Establishing the pipeline DAG

Before experimenting, a dvc pipeline must be established (see the docs if you are new to dvc). Review the contents of `dvc.yaml` below to see the pipeline.

In [1]:
%%bash
cat dvc.yaml

stages:
  download:
    cmd: python download.py
    deps:
    - download.py
    outs:
    - data/MNIST
  train:
    cmd: python train.py
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model.pt:
        checkpoint: true
    metrics:
    - metrics.yaml


The download stage gets the data using the `download.py` script. The train stage performs model training and evaluation on the downloaded data using the `train.py` script. The train stage uses the lr and weight_decay metrics defined in `params.yaml`. The model output is saved to `model.pt`, and the metrics are saved to `metrics.yaml`.

Execute the download stage to get the data.

In [3]:
%%bash
dvc repro download

Running stage 'download' with command:
	python download.py
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock data/.gitignore
Use `dvc push` to send your updates to remote storage.


9920512it [00:02, 3630856.70it/s]                             
  0%|          | 0/28881 [00:00<?, ?it/s]
0it [00:00, ?it/s]
 86%|████████▋ | 1425408/1648877 [00:00<00:00, 2570201.91it/s]

0it [00:00, ?it/s]/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729002806/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
1654784it [00:01, 1553014.84it/s]                             
8192it [00:00, 35645.69it/s]
32768it [00:01, 27713.95it/s]            

**IMPORTANT:** Be sure to run the `git add` command above and also `git commit` before running experiments. Anytime you modify the pipeline, be sure to `dvc repro` and track changes with git before running experiments.

In [1]:
%%bash
git add dvc.lock data/.gitignore
git commit -m "download data"

[exp 9fb284f] download data
 2 files changed, 12 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 dvc.lock


### Run an experiment

Run an experiment with the default parameters defined in `params.yaml`.

In [2]:
%%bash
dvc exp run

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration '656cb11'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'b8d3628'.
Reproduced experiment 'b8d3628'.


Review the output of the run, including identifying hashes, metrics, and parameters:

In [3]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment  ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace   │ -        │ 0.1998 │ 2.2452 │ 0.001 │ 0            │
│ exp         │ 04:05 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ b8d3628 │ 04:05 PM │ 0.1998 │ 2.2452 │ 0.001 │ 0            │
│ ├─╨ 656cb11 │ 04:05 PM │ 0.1216 │ 2.2836 │ 0.001 │ 0            │
└─────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Note that two experiments were run. These are checkpoints for the run. It's not necessary to have checkpoints for experiments, but they can be helpful for models that may be run for a number of epochs. See below for more information about how checkpoints work.

### Experimenting with different parameters

Experiments can be run and compared with different parameters.

In [4]:
%%bash
dvc exp run --params weight_decay=0.1

Stage 'download' didn't change, skipping
Running stage 'train' with command:
	python train.py
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'bbe9fde'.
Updating lock file 'dvc.lock'
Checkpoint experiment iteration 'dcae795'.
Reproduced experiment 'dcae795'.


In [5]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment  ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace   │ -        │  0.098 │ 2.3037 │ 0.001 │ 0.1          │
│ exp         │ 04:05 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ dcae795 │ 04:06 PM │  0.098 │ 2.3037 │ 0.001 │ 0.1          │
│ ├─╨ bbe9fde │ 04:06 PM │  0.098 │ 2.3042 │ 0.001 │ 0.1          │
│ │ ╓ b8d3628 │ 04:05 PM │ 0.1998 │ 2.2452 │ 0.001 │ 0            │
│ ├─╨ 656cb11 │ 04:05 PM │ 0.1216 │ 2.2836 │ 0.001 │ 0            │
└─────────────┴──────────┴────────┴────────┴───────┴──────────────┘


Increasing `weight_decay` didn't help, so revert back to original parameters:

In [9]:
%%bash
git checkout params.yaml

Experiments can also be added in bulk to the queue and executed on demand (see the `-j` flag for parallel execution!).

In [6]:
%%bash
dvc exp run --params lr=0.01 --queue
dvc exp run --params lr=0.1 --queue

Queued experiment '1eaa422' for future execution.
Queued experiment 'f1e4f3d' for future execution.


In [7]:
%%bash
dvc exp show

┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment   ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace    │ -        │  0.098 │ 2.3037 │ 0.001 │ 0.1          │
│ exp          │ 04:05 PM │      - │      - │ 0.001 │ 0            │
│ │ ╓ dcae795  │ 04:06 PM │  0.098 │ 2.3037 │ 0.001 │ 0.1          │
│ ├─╨ bbe9fde  │ 04:06 PM │  0.098 │ 2.3042 │ 0.001 │ 0.1          │
│ │ ╓ b8d3628  │ 04:05 PM │ 0.1998 │ 2.2452 │ 0.001 │ 0            │
│ ├─╨ 656cb11  │ 04:05 PM │ 0.1216 │ 2.2836 │ 0.001 │ 0            │
│ ├── *f1e4f3d │ 04:06 PM │      - │      - │ 0.1   │ 0            │
│ └── *1eaa422 │ 04:06 PM │      - │      - │ 0.01  │ 0            │
└──────────────┴──────────┴────────┴────────┴───────┴──────────────┘


In [8]:
%%bash
dvc exp run --run-all

ERROR: Failed to reproduce experiment 'b20b45f' - Stage: '../tmpdk7tvx0m/dvc.yaml:download'
ERROR: Failed to reproduce experiment '1eaa422' - Stage: '../tmpdk7tvx0m/dvc.yaml:download'
ERROR: Failed to reproduce experiment '6f38ac3' - Stage: '../tmpicrftv3d/dvc.yaml:download'
--- Logging error ---
Traceback (most recent call last):
  File "/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/dvc/logger.py", line 134, in emit
    msg = self.format(record)
  File "/home/dave/.conda/envs/dvc/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/dvc/logger.py", line 94, in format
    cause = ": ".join(_iter_causes(record.exc_info[1]))
  File "/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/dvc/logger.py", line 155, in _iter_causes
    yield str(exc)
  File "/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/dvc/stage/__init__.py", line 166, in __repr__
    return f"Stage: '{self.addre

CalledProcessError: Command 'b'dvc exp run --run-all\n'' returned non-zero exit status 1.